In [2]:
# AVL Tree is a self-balancing binary search tree

# like a binary search tree, the value of each node must be greater than (or equal to) any node in its left subtree
# and less than (or equal to) any node in its right subtree

# the difference is AVL trees keep track of an extra piece of information called the balance factor
# the balance factor of a node is the difference between the heights of the left and right subtree of that node

# balance factor examples:

#        0          2      -2           2      -2         -1          0             -1        
#       / \        /         \         /         \        / \        / \            / \
#      0   0      1          -1      -1           1      1   2      1   0          1  -2
#                /             \       \         /      /   /      /   / \        /     \
#               0               0       0       0      0  -1      0   0   0      0      -1
#                                                           \                             \
#                                                            0                             0

# every node in an AVL tree must maintain a balance factor of -1, 0 or 1
# every time an insertion or deletion is done, the tree will check the balance factors of nodes affected by the operation
# if any node's balance factor has gone above 1 (left heavy) or below -1 (right heavy), the tree will automatically do a rotation to restore balance
# there are 4 types of rotation the tree will choose from, depending on which one will properly fix the imbalance: left, right, left-right, or right-left

# rotation examples:

#     -2                          
#       \
#       -1    -left rotate->    0    
#         \                    / \
#          0                  0   0

#          2                         
#         /
#        1    -right rotate->    0    
#       /                       / \
#      0                       0   0

#          2                         
#         /
#       -1    -left-right rotate->    0    
#         \                          / \
#          0                        0   0

#     -2                         
#       \
#        1    -right-left rotate->    0    
#       /                            / \
#      0                            0   0

# insertion and deletion are more complex with AVL trees because of need to update heights, balance factors, and possibly do rotations with each insertion/deletion
# the advantage is, this makes searching much more efficient because the tree stays strictly balanced.
# with non-self-balancing binary search trees, there is the danger that the tree/subtree(s) will become very unbalanced
# and thus time complexity of search can slow down to O(N) in the worst case of target node being the leaf of a degenerate tree 
# with AVL trees, the strict balancing ensures search time complexity will never be slower than O(logN) even in the worst case

# space complexity: O(N)
# time complexity (of search): O(logN) average case
#							   O(logN) worst case 
#							   O(1) best case (target is root node)

class Node: 
	def __init__(self, data, parent=None): # 1 argument passed creates root node (no parent)
										   # 2 arguments passed creates node and links to parent 
		self.data = data # node holds data 
		self.left = None # links down to left and right child nodes
		self.right = None
		self.parent = parent # link up to parent node
		self.height = 0 # height of node will be recorded here
		self.balance = 0 # balance factor will be recorded here

	def update_heights(self): # update height of current node, then climb up to root, updating heights of current node's ancestors  
		self.height = self.set_height() # call '.set_height()' on current node, setting it's 'height' attribute to the returned value
		climbing = True # 'climbing' flag, = True means climb hasn't reached root node yet
		while climbing == True: # while climbing flag True:
			if self.parent == None: # if current node has no parent (aka is root)...
				climbing = False # set climbing flag to False
			else: # but if current node does have parent (aka isn't root)...
				self = self.parent # set parent as current node 
				self.height = self.set_height() # call '.set_height()' on current node, setting it's 'height' attribute to the returned value

	def set_height(self): # set height of current node 
		if self.left != None and self.right != None: # if current node has both a left and right child...
			return 1 + max(self.left.height,self.right.height) # return 1 + the height of whichever child's 'height' atrribute is greater
		elif self.left != None: # but if current node only has a left child...
			return 1 + self.left.height # return 1 + the height of the left child 
		elif self.right != None: # but if current node only has a right child...
			return 1 + self.right.height # return 1 + the height of the right child
		else: # otherwise, current node has no children
			return 0 # so return 0

	def update_balances(self): # update balance of current node, then climb up to root, updating balances of current node's ancestors
		self.balance = self.set_balance() # call '.set_balance()' on current node, setting it's 'balance' attribute to the returned value
		climbing = True # 'climbing' flag, = True means climb hasn't reached root node yet
		while climbing == True: # while climbing flag True:
			if self.parent == None: # if current node has no parent (aka is root)...
				climbing = False # set climbing flag to False
			else: # but if current node does have parent (aka isn't root)...
				self= self.parent # set parent as current node 
				self.balance = self.set_balance() # call '.set_balance()' on current node, setting it's 'balance' attribute to the returned value

	def set_balance(self): # set balance of current node
		if self.left != None and self.right != None: # if current node has both a left and right child...
			return self.left.height - self.right.height # return the height of its left child - the height of its right child
		elif self.left != None: # but if current node only has a left child...
			return self.height # return the height of its left child
		elif self.right != None: # but if current node only has a right child...
			return 0 - self.height # return 0 - the height of its right child
		else: # otherwise current node has no children 
			return 0 # so return 0

	def insertion_rotations(self,new): # check if newly inserted node has caused any of its ancestors to become unbalanced
									   # if so call the correct rotation to balance them
		if self.balance > 1: # if current node balance is > 1 (left heavy)...
			print('  tree unbalanced at node ' + str(self.data) + '...')
			if new < self.left.data: # and the newly inserted node is < current node's left child...
				print('    *right rotating to fix*')
				print()
				return self.right_rotate() # call 'right_rotate()' on current node.
			else: # but if newly inserted node is >= current node's left child...
				print('    *left-right rotating to fix*')
				print()
				return self.left_right_rotate() # call 'left_right_rotate() on current node.
		elif self.balance < -1: # if current node balance is < -1 (right heavy)...
			print('  tree unbalanced at node ' + str(self.data) + '...')
			if new > self.right.data: # and the newly inserted node is > current node's right child...
				print('    *left rotating to fix*')
				print()
				return self.left_rotate() # call 'left_rotate()' on current node.
			else: # but if newly inserted node is <= current node's right child...
				print('    *right-left rotating to fix*')
				print()
				return self.right_left_rotate() # call 'right_left_rotate() on current node.
		if self.parent != None: # after checking if current node is unbalanced, check if it has a parent, if it does...
			self = self.parent # set parent node as current node...
			self.insertion_rotations(new) # and call 'insertion_rotations()' on current node 
	
	def deletion_rotations(self): # check if node deletion has caused any of deleted node's (or its replacement node's) ancestors to become unbalanced
								  # if so call the correct rotation to balance them
		if self.balance > 1: # if current node balance is > 1 (left heavy)...
			print('  tree unbalanced at node ' + str(self.data) + '...')
			if self.left.balance >= 0: # and its left child's balance is >= 0....
				print('    *right rotating to fix*')
				print()
				return self.right_rotate() # call 'right_rotate()' on current node.
			else: # but if it's left child's balance is < 0:
				print('    *left-right rotating to fix*')
				print()
				return self.left_right_rotate() # call 'left_right_rotate()' on current node.
		elif self.balance < -1: # if current node balance is < -1 (right heavy)...
			print('  tree unbalanced at node ' + str(self.data) + '...')
			if self.right.balance <= 0: # and its right child node is <= 0...
				print('    *left rotating to fix*')
				print()
				return self.left_rotate() # call 'left_rotate()' on current node.
			else: # but if its right child balance is > 0...
				print('    *right-left rotating to fix*')
				print()
				return self.right_left_rotate()	# call 'right_left_rotate()' on current node.
		if self.parent != None: # after checking if current node is unbalanced, check if it has a parent, if it does...
			self = self.parent # set parent node as current node...
			self.deletion_rotations() # and call 'insertion_rotations()' on current node 
		return self

	def left_rotate(self): # left rotate unbalanced (sub)tree 
		pivot = self.right # set current node's right child as pivot
		if self.parent != None: # if current node has a parent...
			if self.parent.left == self: # and current node is its parent's left child...
				self.parent.left = pivot # set the pivot as the parent's new left child.
			else: # or if current node is its parent's right child...
				self.parent.right = pivot # set the pivot as the parent's new right child.
		pivot.parent = self.parent # set current node's parent (if any) as the pivot's parent
		self.parent = pivot # set the pivot as the current node's parent
		self.right = pivot.left # set the pivot's left child (if any) as the current node's right child
		pivot.left = self # set the current node as the pivot's left child
		self.update_heights() # call 'update_heights()' Node method from current node (updating heights now that rotation complete)
		self.update_balances() # call 'update_balances()' Node method from current node (updating balances now that rotation complete)
			
	def right_rotate(self): # right rotate unbalanced (sub)tree
		pivot = self.left # set current node's left child as pivot
		if self.parent != None: # if current node has a parent...
			if self.parent.left == self: # and current node is its parent's left child...
				self.parent.left = pivot # set the pivot as the parent's new left child.
			else: # or if current node is its parent's right child...
				self.parent.right = pivot # set the pivot as the parent's new right child.
		pivot.parent = self.parent # set current node's parent (if any) as the pivot's parent
		self.parent = pivot # set the pivot as the current node's parent
		self.left = pivot.right # set the pivot's right child (if any) as the current node's left child
		pivot.right = self # set the current node as the pivot's right child
		self.update_heights() # call 'update_heights()' Node method from current node (updating heights now that rotation complete)
		self.update_balances() # call 'update_balances()' Node method from current node (updating balances now that rotation complete)

	def left_right_rotate(self): # left-right rotate unbalanced (sub)tree
		self = self.left # set current node's left child as current node 
		self.left_rotate() # call 'left_rotate()' on current node
		self = self.parent.parent # after left rotation complete, set current node's grandparent as current node
		self.right_rotate() # call 'right_rotate()' on current node

	def right_left_rotate(self): # right-left rotate unbalanced (sub)tree
		self = self.right # set current node's right child as current node
		self.right_rotate() # call 'right_rotate()' on current node
		self = self.parent.parent # after right rotation complete, set current node's grandparent as current node
		self.left_rotate() # call 'left_rotate()' on current node

class Tree:
	def __init__(self):
		self.root = None # holds root node 
		self.curr = None # holds current (selected) node
		print()
		print("*new tree*")
		print()

	def update_root(self): # update root of tree if needed after any rotations have been done
		while True: # while loop continues until break called 
			if self.curr.parent == None: # if current node doesn't have a parent...
				self.root = self.curr # means it's the root, set Tree's 'root' attribute to current node
				break # and break out of loop.
			else: # if current node does have a parent...
				self.curr = self.curr.parent # set parent as current node, continue loop
			
	def insert(self, new): # insert new node
		if self.curr == None: # if no current node, means tree is empty
			self.curr = Node(new) # argument passed to 'Node' class to create new node with 'new' as data
			self.root = self.curr # set new node as root
			print( "*inserting " + str(new) + "*")
			print("  ...as root")
			print()
		else: # if there is a current node...
			if new < self.curr.data:  # compare new node to current node, if the new node is lower...
				if self.curr.left == None: # and the current node has no left child...
					self.curr.left = Node(new, self.curr) # create new node with 'new' data
														  # linked to current node as parent
														  # set as current node's left child
					print( "*inserting " + str(new) + "*")
					print("  ... as left child of " + str(self.curr.data))
					print()
				else: # but if the current node already has a left child...
					self.curr = self.curr.left # select left child as the current node
					self.insert(new) # and call 'insert()' recursively
			else: # but if the new node is greater...
				if self.curr.right == None: # and the current node has no right child...
					self.curr.right = Node(new, self.curr) # create new node with 'new' data
														   # linked to current node as parent
														   # set as current node's right child
					print( "*inserting " + str(new) + "*")
					print("  ... as right child of " + str(self.curr.data))
					print()
				else: # but if the current node already has a right child...
					self.curr = self.curr.right # select right child as the current node
					self.insert(new) # and call 'insert()' recursively
		self.curr.update_heights() # call 'update_heights()' Node method from current node
		self.curr.update_balances() # call 'update_balances()' Node method from current node
		self.curr.insertion_rotations(new) # call 'insertion_rotations()' Node method from current node, pass new node's data
		self.update_root() # call 'update_root' Tree method 
		self.curr = self.root # finally, select the root as the current node (back to top of tree for next function call)

	def delete(self, target): # delete target 
		if self.curr == None: # if current node empty...
			print("no " + str(target) + " to delete") # either tree is empty or reached end of search path without finding target 
			print()
			self.curr = self.root # select root as the current node (back to top of tree for next function call)
			return
		if self.curr.data == target: # if current node == target, target for deletion found...
			if self.curr.left == None and self.curr.right == None: # if target node has no children...
				if self.curr.parent != None: # and isn't the root...
					parent = self.curr.parent # hold parent of node being deleted
					if parent.left == self.curr: # if target node is left child of its parent...
						parent.left = None # set its parent's left child to None (breaking the link)
					elif parent.right == self.curr: # or if it's the right child of its parent...
						parent.right = None # set its parent's right child to None (breaking the link)
				elif self.curr.parent == None: # if it is the root...
					self.root = None # set root to None
				print("*deleting " + str(target) + "*")					
				print()
			elif self.curr.left != None and self.curr.right == None: # if target node has only left child...
				replacement = self.curr.left # hold left child as replacement for target being deleted			
				self.curr.data = replacement.data # set its node data to its replacement's data (like moving replacement up to take its place)
				self.curr.right = replacement.right # set right link to replacement's right child node (like skipping over replacement which was moved up)
				self.curr.left = replacement.left # set left link to replacement's left child node (like skipping over replacement which was moved up)
				if self.curr.left != None: # if now has a left child:
					self.curr.left.parent = self.curr # set left child's parent as it (completing link in both directions)
				if self.curr.right != None: # if now has a right child:
					self.curr.right.parent = self.curr # set right child's parent as it (completeing link in both directions)
				print("*deleting " + str(target) + "*")
				print("  ... replaced by " + str(self.curr.data))
				print()
			elif self.curr.left == None and self.curr.right != None: # if target node has only right child...
				replacement = self.curr.right # hold right child as replacement for target being deleted	
				self.curr.data = replacement.data # set its node data to its replacements's data (like moving replacement up to take its place)
				self.curr.left = replacement.left # set left link to replacements's left child node (like skipping over replacement which was moved up)
				self.curr.right = replacement.right # set right link to replacements's right child node (like skipping over replacement which was moved up)
				if self.curr.left != None: # if now has a left child:
					self.curr.left.parent = self.curr # set left child's parent as it (completing link in both directions)
				if self.curr.right != None: # if now has a right child:
					self.curr.right.parent = self.curr # set right child's parent as it (completeing link in both directions)
				print("*deleting " + str(target) + "*")
				print("  ... replaced by " + str(self.curr.data))
				print()
			elif self.curr.left != None and self.curr.right != None: # if target node has two children...
																	 # target node should be replaced with its inorder successor (next largest node in tree)
																	 # the leftmost child of the target node's right subtree is the inorder successor. that means:
																	 # if the target nodes 's right child doesn't have a left child of its own, it's the inorder successor
																	 # if target node's right child does have a left child of its own...
																	 # follow that child's left path as far down as possible. the last left child in the path is the inorder successor 
				node_to_delete = self.curr # hold node being deleted 
				self.curr = self.curr.right # select right child as current node
				while self.curr.left != None: # while current node has a left child...
					self.curr = self.curr.left # select the left child as current node. this will lead to target's inorder successor being selected as current node
				replacement = self.curr # hold replacement
				node_to_delete.data = replacement.data # set node being deleted's data to its replacement's data (like moving replacement up to take its place)
				print("*deleting " + str(target) + "*")	
				print("  ... replaced by " + str(replacement.data))
				print()	
				if replacement.left == None and replacement.right == None: # if replacement node is a leaf (no children)...
					parent = replacement.parent # hold parent of replacement
					if parent.left == replacement: # if replacement is left child of its parent...
						parent.left = None # set its parent's left child to None (like breaking the link)
					elif parent.right == replacement: # or if it's the right child of its parent...
						parent.right = None # set it's parent's right child to None (like breaking the link)
				elif replacement.right != None: # but if replacement node has right child...
					replacement.data = replacement.right.data # set its data to its right child's data (like shifting right child up to take its place)
					replacement.right = replacement.right.right # set its right link to its child's right link (like skipping over it)
					if replacement.right != None: # if it now links to a right child...
						replacement.right.parent = replacement # set right child's parent as it (completing link in both directions)
			self.curr.update_heights() # call 'update_heights()' Node method from current node
			self.curr.update_balances() # call 'update_balances()' Node method from current node
			self.curr.deletion_rotations() # call 'deletion_rotations()' Node method from current node
			self.update_root() # call 'update_root' Tree method 
			self.curr = self.root # select root as the current node (back to top of tree for next function call)
			return
		if target < self.curr.data: # if target is lower than current node...
			self.curr = self.curr.left # select left child as current node
			self.delete(target) # and call 'delete()' recursively
		else: # if target is greater than current node...
			self.curr = self.curr.right # select right child as current node
			self.delete(target) # and call 'delete()' recursively

	def info(self): # display tree info to user 
		print('root: ' + str(self.curr.data))
		if self.curr.left != None:
			print("root's left child: " + str(self.curr.left.data))
		if self.curr.right != None:
			print("root's right child: " + str(self.curr.right.data))
		print('root balance: ' + str(self.curr.balance))
		if self.curr.left != None:
			print('left subtree balance:' + str(self.curr.left.balance))
		if self.curr.right != None:
			print('right subtree balance:' + str(self.curr.right.balance))
		print()

''' # search not used in this example but works the same as in a non-AVL Binary Search Tree 
	def search(self, target): # search tree for target
		if self.curr == None: # if current node empty... 
			print(str(target) + " not found") # either tree is empty or reached end of search path without finding target 
			print()
			self.curr = self.root # select root as the current node (back to top of tree for next function call)
			return 
		if self.curr.data == target: # if current node == target, target found!...
			print(str(target) + " found") # print location information (target node's parent, left child, right child)
			if self.curr == self.root: 
				print("  parent: None")
			else:
				print("  parent: " + str(self.curr.parent.data))
			if self.curr.left == None:
				print("  left child: None")
			else:
				print("  left child: " + str(self.curr.left.data))
			if self.curr.right == None:
				print("  right child: None")
			else:
				print("  right child: " + str(self.curr.right.data))
			print()
			self.curr = self.root # select root as the current node (back to top of tree for next function call)
			return
		if target < self.curr.data: # if target is lower than current node...
			self.curr = self.curr.left # select left child as current node
			self.search(target) # and call 'search()' recursively 
		else: # if target is greater than current node...
			self.curr = self.curr.right # select right child as current node
			self.search(target) # and call 'search()' recursively 
''' ;

# code and comments by github.com/alandavidgrunberg


In [5]:
t = Tree()
t.insert(1)
t.insert(2)
t.info()
t.insert(3)
t.info()

#      1                          
#       \
#        2    -left rotate->    2    
#         \                    / \
#          3                  1   3



*new tree*

*inserting 1*
  ...as root

*inserting 2*
  ... as right child of 1

root: 1
root's right child: 2
root balance: -1
right subtree balance:0

*inserting 3*
  ... as right child of 2

  tree unbalanced at node 1...
    *left rotating to fix*

root: 2
root's left child: 1
root's right child: 3
root balance: 0
left subtree balance:0
right subtree balance:0



In [6]:
t = Tree()
t.insert(3)
t.insert(2)
t.info()
t.insert(1)
t.info()

#          3                         
#         /
#        2    -right rotate->    2    
#       /                       / \
#      1                       1   3



*new tree*

*inserting 3*
  ...as root

*inserting 2*
  ... as left child of 3

root: 3
root's left child: 2
root balance: 1
left subtree balance:0

*inserting 1*
  ... as left child of 2

  tree unbalanced at node 3...
    *right rotating to fix*

root: 2
root's left child: 1
root's right child: 3
root balance: 0
left subtree balance:0
right subtree balance:0



In [7]:
t = Tree()
t.insert(3)
t.insert(1)
t.insert(2)
t.info()

#          3                         
#         /
#        1    -left-right rotate->    2    
#         \                          / \
#          2                        1   3



*new tree*

*inserting 3*
  ...as root

*inserting 1*
  ... as left child of 3

*inserting 2*
  ... as right child of 1

  tree unbalanced at node 3...
    *left-right rotating to fix*

root: 2
root's left child: 1
root's right child: 3
root balance: 0
left subtree balance:0
right subtree balance:0



In [8]:
t = Tree()
t.insert(1)
t.insert(3)
t.insert(2)
t.info()

#      1                         
#       \
#        3    -right-left rotate->    2    
#       /                            / \
#      2                            1   3



*new tree*

*inserting 1*
  ...as root

*inserting 3*
  ... as right child of 1

*inserting 2*
  ... as left child of 3

  tree unbalanced at node 1...
    *right-left rotating to fix*

root: 2
root's left child: 1
root's right child: 3
root balance: 0
left subtree balance:0
right subtree balance:0



In [9]:
t = Tree()
t.insert(4)
t.insert(3)
t.insert(7)
t.insert(2)
t.insert(5)
t.info()
t.insert(6)
t.info()

#          4                                4
#         / \                              / \
#        3   7                            3   6   
#       /   /                            /   / \
#      2   5    -left-right rotate->    2   5   7   
#           \
#            6   



*new tree*

*inserting 4*
  ...as root

*inserting 3*
  ... as left child of 4

*inserting 7*
  ... as right child of 4

*inserting 2*
  ... as left child of 3

*inserting 5*
  ... as left child of 7

root: 4
root's left child: 3
root's right child: 7
root balance: 0
left subtree balance:1
right subtree balance:1

*inserting 6*
  ... as right child of 5

  tree unbalanced at node 7...
    *left-right rotating to fix*

root: 4
root's left child: 3
root's right child: 6
root balance: 0
left subtree balance:1
right subtree balance:0



In [10]:
t.insert(8)

t.delete(4)
t.info()

#          5                          5
#         / \                        / \
#        3   6    -left rotate->    3   7  
#       /     \                    /   / \
#      2       7                  2   6   8 
#               \
#                8 


*inserting 8*
  ... as right child of 7

*deleting 4*
  ... replaced by 5

  tree unbalanced at node 6...
    *left rotating to fix*

root: 5
root's left child: 3
root's right child: 7
root balance: 0
left subtree balance:1
right subtree balance:0



In [11]:
t.delete(7)
t.delete(3)
t.delete(2)
t.info()

#         5                          
#          \
#           8    -right-left rotate->    6 
#          /                            / \
#         6                            5   8 


*deleting 7*
  ... replaced by 8

*deleting 3*
  ... replaced by 2

*deleting 2*

  tree unbalanced at node 5...
    *right-left rotating to fix*

root: 6
root's left child: 5
root's right child: 8
root balance: 0
left subtree balance:0
right subtree balance:0

